In [1]:
from pyspark import SparkConf, SparkContext

spark_conf = SparkConf().setAppName("spark-context-test")
spark = SparkContext(conf = spark_conf)
SparkConf().getAll()

24/08/22 08:05:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


[('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.name', 'spark-context-test'),
 ('spark.executor.extraJavaOptions',
  '-Dio.netty.tryReflectionSetAccessible=true'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.extraJavaOptions',
  '-Dio.netty.tryReflectionSetAccessible=true')]

In [2]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .master("local").appName("DataProcessing")
    .getOrCreate())

spark

In [3]:
%%bash
head -n 10 ../data/flights.csv

flight,time,wind_speed,wind_angle,battery_voltage,battery_current,position_x,position_y,position_z,orientation_x,orientation_y,orientation_z,orientation_w,velocity_x,velocity_y,velocity_z,angular_x,angular_y,angular_z,linear_acceleration_x,linear_acceleration_y,linear_acceleration_z,speed,payload,altitude,date,time_day,route
1,0.0,0.100000001,12.0,24.22217369,0.087469511,-79.78239570000002,40.45804724,269.3324017,0.001772306,0.005609551,-0.255525738,0.966784358,-0.012825926,-0.005644709,0.010474564,-0.0008251889999999999,0.0014074279999999998,0.006815216999999999,0.004257701,-0.120404978,-9.811136699,4.0,0.0,25,2019-04-07,10:13,R5
1,0.2,0.100000001,3.0,24.22718048,0.095421284,-79.78239587,40.45804729,269.332056,0.001767616,0.005606697,-0.255530089,0.966783345,-0.015409501,-0.001635618,0.010136867,1.24e-06,-0.002861936,0.002033538,0.006174664,-0.116396593,-9.810391763,4.0,0.0,25,2019-04-07,10:13,R5
1,0.3,0.100000001,352.0,24.22592926,0.095421284,-79.78239589,40.4580473,269.3330812,0.001

In [4]:
# data 타입 표시
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, DoubleType, TimestampType

In [5]:
csv_file_path="../data/flights.csv"
df = spark.read.csv(csv_file_path,header=True,inferSchema=True)

In [6]:
# Schema 정의
schema = StructType([
    StructField("time", DoubleType(), True),
    StructField("battery_voltage", DoubleType(), True),
    StructField("battery_current", DoubleType(), True),
    StructField("altitude", DoubleType(), True), 
    StructField("route", StringType(), True),
    StructField("date", TimestampType(), True),
    StructField("time_day", TimestampType(), True)
])

In [7]:
df.printSchema()
df.show(5)

24/08/22 08:06:05 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


root
 |-- flight: integer (nullable = true)
 |-- time: double (nullable = true)
 |-- wind_speed: double (nullable = true)
 |-- wind_angle: double (nullable = true)
 |-- battery_voltage: double (nullable = true)
 |-- battery_current: double (nullable = true)
 |-- position_x: double (nullable = true)
 |-- position_y: double (nullable = true)
 |-- position_z: double (nullable = true)
 |-- orientation_x: double (nullable = true)
 |-- orientation_y: double (nullable = true)
 |-- orientation_z: double (nullable = true)
 |-- orientation_w: double (nullable = true)
 |-- velocity_x: double (nullable = true)
 |-- velocity_y: double (nullable = true)
 |-- velocity_z: double (nullable = true)
 |-- angular_x: double (nullable = true)
 |-- angular_y: double (nullable = true)
 |-- angular_z: double (nullable = true)
 |-- linear_acceleration_x: double (nullable = true)
 |-- linear_acceleration_y: double (nullable = true)
 |-- linear_acceleration_z: double (nullable = true)
 |-- speed: double (nullable

In [8]:
csv_file_path = "../data/flights.csv"
df = spark.read.csv(csv_file_path,header=True, schema=schema)

In [9]:
df.printSchema()
df.show(5)

root
 |-- time: double (nullable = true)
 |-- battery_voltage: double (nullable = true)
 |-- battery_current: double (nullable = true)
 |-- altitude: double (nullable = true)
 |-- route: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- time_day: timestamp (nullable = true)

+----+---------------+---------------+--------+-----------+----+--------+
|time|battery_voltage|battery_current|altitude|      route|date|time_day|
+----+---------------+---------------+--------+-----------+----+--------+
| 1.0|            0.0|    0.100000001|    12.0|24.22217369|null|    null|
| 1.0|            0.2|    0.100000001|     3.0|24.22718048|null|    null|
| 1.0|            0.3|    0.100000001|   352.0|24.22592926|null|    null|
| 1.0|            0.5|    0.100000001|   354.0|24.22467804|null|    null|
| 1.0|            0.6|    0.100000001|   359.0|24.21090508|null|    null|
+----+---------------+---------------+--------+-----------+----+--------+
only showing top 5 rows



24/08/22 08:06:05 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 28, schema size: 7
CSV file: file:///home/jovyan/data/flights.csv


In [10]:
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import col, to_timestamp, concat, lit

In [11]:
df.printSchema()

root
 |-- time: double (nullable = true)
 |-- battery_voltage: double (nullable = true)
 |-- battery_current: double (nullable = true)
 |-- altitude: double (nullable = true)
 |-- route: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- time_day: timestamp (nullable = true)



In [12]:
df.select("date", "time_day").show(5)

+----+--------+
|date|time_day|
+----+--------+
|null|    null|
|null|    null|
|null|    null|
|null|    null|
|null|    null|
+----+--------+
only showing top 5 rows



24/08/22 08:06:07 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: battery_current, position_x
 Schema: date, time_day
Expected: date but found: battery_current
CSV file: file:///home/jovyan/data/flights.csv


In [13]:
df = df.withColumn("full_timestamp", 
                   to_timestamp(concat(col("date").cast("string"), 
                                       lit(" "), 
                                       col("time_day")), 
                                "yyyy-MM-dd HH:mm"))

print("\nNew column 'full_timestamp' added. Updated schema:")
df.printSchema()
print("\nSample data with new column:")
df.show(5)


New column 'full_timestamp' added. Updated schema:
root
 |-- time: double (nullable = true)
 |-- battery_voltage: double (nullable = true)
 |-- battery_current: double (nullable = true)
 |-- altitude: double (nullable = true)
 |-- route: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- time_day: timestamp (nullable = true)
 |-- full_timestamp: timestamp (nullable = true)


Sample data with new column:
+----+---------------+---------------+--------+-----------+----+--------+--------------+
|time|battery_voltage|battery_current|altitude|      route|date|time_day|full_timestamp|
+----+---------------+---------------+--------+-----------+----+--------+--------------+
| 1.0|            0.0|    0.100000001|    12.0|24.22217369|null|    null|          null|
| 1.0|            0.2|    0.100000001|     3.0|24.22718048|null|    null|          null|
| 1.0|            0.3|    0.100000001|   352.0|24.22592926|null|    null|          null|
| 1.0|            0.5|    0.100000001|  

24/08/22 08:06:07 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 28, schema size: 7
CSV file: file:///home/jovyan/data/flights.csv


In [14]:
selected_columns = ["full_timestamp", "time", "battery_voltage", "battery_current", "altitude", "route"]
df_selected = df.select(selected_columns)

print("\nSelected columns. New DataFrame schema:")
df_selected.printSchema()

print("\nSample data of selected columns:")
df_selected.show(5)


Selected columns. New DataFrame schema:
root
 |-- full_timestamp: timestamp (nullable = true)
 |-- time: double (nullable = true)
 |-- battery_voltage: double (nullable = true)
 |-- battery_current: double (nullable = true)
 |-- altitude: double (nullable = true)
 |-- route: string (nullable = true)


Sample data of selected columns:
+--------------+----+---------------+---------------+--------+-----------+
|full_timestamp|time|battery_voltage|battery_current|altitude|      route|
+--------------+----+---------------+---------------+--------+-----------+
|          null| 1.0|            0.0|    0.100000001|    12.0|24.22217369|
|          null| 1.0|            0.2|    0.100000001|     3.0|24.22718048|
|          null| 1.0|            0.3|    0.100000001|   352.0|24.22592926|
|          null| 1.0|            0.5|    0.100000001|   354.0|24.22467804|
|          null| 1.0|            0.6|    0.100000001|   359.0|24.21090508|
+--------------+----+---------------+---------------+--------+-

24/08/22 08:06:07 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 28, schema size: 7
CSV file: file:///home/jovyan/data/flights.csv


In [15]:
# 시간순으로 정렬
df_sorted = df_selected.orderBy("full_timestamp", "time")

print("\nData sorted by time. Sample of sorted data:")
df_sorted.show(5)


Data sorted by time. Sample of sorted data:


24/08/22 08:06:07 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 28, schema size: 7
CSV file: file:///home/jovyan/data/flights.csv


+--------------+----+---------------+---------------+--------+-----------+
|full_timestamp|time|battery_voltage|battery_current|altitude|      route|
+--------------+----+---------------+---------------+--------+-----------+
|          null| 1.0|            0.6|    0.100000001|   359.0|24.21090508|
|          null| 1.0|            0.3|    0.100000001|   352.0|24.22592926|
|          null| 1.0|            0.5|    0.100000001|   354.0|24.22467804|
|          null| 1.0|            0.0|    0.100000001|    12.0|24.22217369|
|          null| 1.0|            0.2|    0.100000001|     3.0|24.22718048|
+--------------+----+---------------+---------------+--------+-----------+
only showing top 5 rows



In [16]:
# 시각화를 위해 Pandas DataFrame으로 변환
pandas_df = df_sorted.toPandas()
print("\nConverted to Pandas DataFrame. Shape:", pandas_df.shape)

24/08/22 08:06:09 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 28, schema size: 7
CSV file: file:///home/jovyan/data/flights.csv
24/08/22 08:06:10 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 28, schema size: 7
CSV file: file:///home/jovyan/data/flights.csv



Converted to Pandas DataFrame. Shape: (257896, 6)


In [17]:
# 시각화 함수
def visualize_data(df):
    fig, axs = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle("Drone Data Visualization")
    
    sns.lineplot(x="time", y="battery_voltage", hue="route", data=df, ax=axs[0, 0])
    axs[0, 0].set_title("Battery Voltage over Time")
    
    sns.lineplot(x="time", y="battery_current", hue="route", data=df, ax=axs[0, 1])
    axs[0, 1].set_title("Battery Current over Time")
    
    sns.scatterplot(x="time", y="altitude", hue="route", data=df, ax=axs[1, 0])
    axs[1, 0].set_title("Altitude over Time")
    
    sns.boxplot(x="route", y="battery_voltage", data=df, ax=axs[1, 1])
    axs[1, 1].set_title("Battery Voltage Distribution by Route")
    
    plt.tight_layout()
    plt.show()